In [19]:
import boto3
import pprint
from pprint import pprint
import copy

In [2]:
print(boto3.__version__)

1.35.96


In [3]:
# first we will find out which user or role is being used by sagemaker
# We will assign BedrockFUllAccess to it. 
session = boto3.Session()
sts_client = session.client('sts')
identity = sts_client.get_caller_identity()
print(identity['Arn'])

arn:aws:sts::794038231401:assumed-role/SageMaker-ExecutionRole-20250103T203496/SageMaker


In [4]:
# import kb_id from my private file config.py
from config import *

In [5]:
kb_id = kb_id
region_name = 'us-east-1'
model_id =  ["Claude 3 Sonnet", "anthropic.claude-3-sonnet-20240229-v2:0"]
model_arn = 'arn:aws:bedrock:us-east-1::anthropic.claude-3-sonnet-20240620-v1'

In [6]:
bedrock = boto3.client(service_name='bedrock')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name = region_name)

In [26]:
# we will create a small retrieveAndGenerateConfiguration that I know works
retrieveAndGenerateConfiguration={
    'type': 'KNOWLEDGE_BASE',
    'knowledgeBaseConfiguration': {
        'knowledgeBaseId': kb_id,
        'modelArn': model_arn,
        'generationConfiguration': {
            'promptTemplate': {
                'textPromptTemplate': '''<instruction>Here are the search results from the knowledge base: $search_results$
                Study these search results carefully and identify the different question formats present. Choose one of those formats and use it to formulate your next question. Respond with the question you formulated, without any additional preamble or explanation. Answer as accurately as possible!</instruction>'''
            }
        },
    }
}
query = 'list repeated packages, associate them with their source file'

In [27]:
error1_config = copy.deepcopy(retrieveAndGenerateConfiguration)
e1_model_arns = ['arn:aws:bedrock:us-east-1::anthropic.claude-3-sonnet-20240620-v1','arn:aws:bedrock:us-east-1::anthropic.claude-3-5-sonnet-20241022-v2:0']


# Add the orchestrationConfiguration
error1_config['knowledgeBaseConfiguration']['orchestrationConfiguration'] = {
    'queryTransformationConfiguration': {
        'type': 'QUERY_DECOMPOSITION',
    }
}

for m in e1_model_arns:
    error1_config['knowledgeBaseConfiguration']['modelArn'] = m
    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration=error1_config
        )
    except Exception as e:
        print(f"Exception occurred for model {m}:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")

Exception occurred for model arn:aws:bedrock:us-east-1::anthropic.claude-3-sonnet-20240620-v1:
Error type: ValidationException
Error message: An error occurred (ValidationException) when calling the RetrieveAndGenerate operation: The model arn provided is not supported when using query decomposition. Please check your configuration and retry the request.
Exception occurred for model arn:aws:bedrock:us-east-1::anthropic.claude-3-5-sonnet-20241022-v2:0:
Error type: ValidationException
Error message: An error occurred (ValidationException) when calling the RetrieveAndGenerate operation: The model arn provided is not supported when using query decomposition. Please check your configuration and retry the request.


In [28]:
# the second error
error2_config = copy.deepcopy(retrieveAndGenerateConfiguration)
for m in e1_model_arns:
    error2_config['knowledgeBaseConfiguration']['modelArn'] = m
    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration=error2_config
        )
    except Exception as e:
        print(f"Exception occurred for model {m}:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")

Exception occurred for model arn:aws:bedrock:us-east-1::anthropic.claude-3-sonnet-20240620-v1:
Error type: ValidationException
Error message: An error occurred (ValidationException) when calling the RetrieveAndGenerate operation: Custom prompt templates must be provided for both Orchestration and Generation when using the selected model. Please recheck the model arn or provide custom prompts and retry the request.
Exception occurred for model arn:aws:bedrock:us-east-1::anthropic.claude-3-5-sonnet-20241022-v2:0:
Error type: ValidationException
Error message: An error occurred (ValidationException) when calling the RetrieveAndGenerate operation: Custom prompt templates must be provided for both Orchestration and Generation when using the selected model. Please recheck the model arn or provide custom prompts and retry the request.


In [29]:
error3_config = copy.deepcopy(retrieveAndGenerateConfiguration)

error3_config['knowledgeBaseConfiguration']['orchestrationConfiguration'] = {
    'queryTransformationConfiguration': {
        'type': 'QUERY_DECOMPOSITION',
        'promptTemplate': {
            'textPromptTemplate': '<instruction>You are a question formulation assistant. Review the conversation history: $conversation_history$ Your task is to: 1. Analyze the conversation context 2. Break down complex queries into simpler components 3. Ensure questions align with the knowledge base content $output_format_instructions$ Generate a focused search query that will help find relevant question formats in the knowledge base.</instruction>'
        }
    }
}

for m in e1_model_arns:
    error2_config['knowledgeBaseConfiguration']['modelArn'] = m
    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration=error3_config
        )
    except Exception as e:
        print(f"Exception occurred for model {m}:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")

Exception occurred for model arn:aws:bedrock:us-east-1::anthropic.claude-3-sonnet-20240620-v1:
Error type: ParamValidationError
Error message: Parameter validation failed:
Unknown parameter in retrieveAndGenerateConfiguration.knowledgeBaseConfiguration.orchestrationConfiguration.queryTransformationConfiguration: "promptTemplate", must be one of: type
Exception occurred for model arn:aws:bedrock:us-east-1::anthropic.claude-3-5-sonnet-20241022-v2:0:
Error type: ParamValidationError
Error message: Parameter validation failed:
Unknown parameter in retrieveAndGenerateConfiguration.knowledgeBaseConfiguration.orchestrationConfiguration.queryTransformationConfiguration: "promptTemplate", must be one of: type


In [28]:
# now I will provide a successful example

In [30]:
# first we correct the model names, Here is a correct list of them
for model in bedrock.list_foundation_models()['modelSummaries']:
    print(model['modelId'])

amazon.titan-tg1-large
amazon.titan-image-generator-v1:0
amazon.titan-image-generator-v1
amazon.titan-image-generator-v2:0
amazon.titan-text-premier-v1:0
amazon.nova-pro-v1:0:300k
amazon.nova-pro-v1:0
amazon.nova-lite-v1:0:300k
amazon.nova-lite-v1:0
amazon.nova-canvas-v1:0
amazon.nova-reel-v1:0
amazon.nova-micro-v1:0:128k
amazon.nova-micro-v1:0
amazon.titan-embed-g1-text-02
amazon.titan-text-lite-v1:0:4k
amazon.titan-text-lite-v1
amazon.titan-text-express-v1:0:8k
amazon.titan-text-express-v1
amazon.titan-embed-text-v1:2:8k
amazon.titan-embed-text-v1
amazon.titan-embed-text-v2:0:8k
amazon.titan-embed-text-v2:0
amazon.titan-embed-image-v1:0
amazon.titan-embed-image-v1
stability.stable-diffusion-xl-v1:0
stability.stable-diffusion-xl-v1
ai21.j2-grande-instruct
ai21.j2-jumbo-instruct
ai21.j2-mid
ai21.j2-mid-v1
ai21.j2-ultra
ai21.j2-ultra-v1:0:8k
ai21.j2-ultra-v1
ai21.jamba-instruct-v1:0
ai21.jamba-1-5-large-v1:0
ai21.jamba-1-5-mini-v1:0
anthropic.claude-instant-v1:2:100k
anthropic.claude-in

In [42]:
# Note that models that end with k usually are for provisioned throughput. Since we are using on-demand we can use the ones that end in 0
models = ['anthropic.claude-3-sonnet-20240229-v1:0']

In [47]:
# a corrected configuration per the boto3 documentation in https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html

# Important!: There is no promptTemplate under queryTransformationConfiguration, but under orchestrationConfiguration
retrieveAndGenerateCorrectedConfiguration = {
    'type': 'KNOWLEDGE_BASE',
    'knowledgeBaseConfiguration': {
        'knowledgeBaseId': kb_id,
        'generationConfiguration': {
            'promptTemplate': {
                'textPromptTemplate': '''<instruction>Here are the search results from the knowledge base: $search_results$
                Study these search results carefully and Answer as accurately as possible!</instruction>'''
            }
        },
        'orchestrationConfiguration': {
            'queryTransformationConfiguration': {
                'type': 'QUERY_DECOMPOSITION',
            },
            'promptTemplate': {
                'textPromptTemplate': '''<instruction>You are an assistant. Review the conversation history: $conversation_history$ Your task is to:
    1. Analyze the conversation context
    2. Break down complex queries into simpler components
    3. Ensure questions align with the knowledge base content
    $output_format_instructions$
    Generate a focused search query that will help find relevant information from the knowledge base.</instruction>'''
            }
        }
    }
}




In [48]:
for m in models:
    retrieveAndGenerateCorrectedConfiguration['knowledgeBaseConfiguration']['modelArn'] = m
    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration=retrieveAndGenerateCorrectedConfiguration
        )
        generated_text = response['output']['text']
        print(generated_text)
    except Exception as e:
        print(f"Exception occurred for model {m}:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")

Based on the search results, here are the repeated packages and their associated source files:

pexpect=4.9.0
- Source 1
- Source 5

pickleshare=0.7.5  
- Source 1
- Source 5  

pillow=11.0.0
- Source 1 
- Source 5

pip=24.3.1
- Source 1
- Source 5

platformdirs=4.3.6
- Source 1
- Source 5

readline=8.2
- Source 1
- Source 3

rubicon-objc=0.4.9
- Source 1 
- Source 3

selenium=4.27.1
- Source 1
- Source 3  

selenium-manager=4.27.0
- Source 1
- Source 3

setuptools=75.6.0
- Source 1
- Source 3

xorg-libsm=1.2.5
- Source 3
- Source 4

xorg-libx11=1.8.10
- Source 3 
- Source 4

xorg-libxau=1.0.12
- Source 3
- Source 4

xorg-libxdmcp=1.1.5  
- Source 3
- Source 4

xorg-libxext=1.3.6
- Source 3
- Source 4

xorg-libxmu=1.2.1
- Source 3
- Source 4

liblapack=3.9.0
- Source 2
- Source 5

libopenblas=0.3.28
- Source 2
- Source 5  

libpng=1.6.44
- Source 2
- Source 5
